In [1]:
import lxml.html
import requests
from lxml import etree
import numpy as np
from tqdm import tqdm_notebook

### Парсинг страниц, создание общего CSV файла с 23 столбцами из задания

In [2]:
ENTRY_POINT_URL = 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227'
page = requests.get(ENTRY_POINT_URL)

tree = lxml.html.fromstring(page.text)

# [(region_name, region_url)]
region_data = []
table = tree.xpath('//table[2]//tr[4]//table[6]//td[2]//a')
for el in table:
    region_data.append((el.text_content(), el.get('href')))
#     TIKS.append(el.text_content())
#     TIK_urls.append(el.get('href'))
region_data_array = np.array(region_data)
# region_data_array[:, 0]

In [3]:
# [(region_name, TIK_name, TIK_url)]
TIK_data = []
for (region_name, region_url) in tqdm_notebook(region_data):
    region_page = requests.get(region_url)
    tree = lxml.html.fromstring(region_page.text)
    
    # /html/body/table[2]/tbody/tr[4]/td/table[6]/tbody/tr/td[2]/div/table/tbody/tr[1]/td[1]/nobr/a
    TIK_elements = tree.xpath('//nobr//a')
    for el in TIK_elements:
        TIK_name = el.text_content()
        TIK_url = el.get('href')
        TIK_data.append((
            region_name,
            TIK_name,
            TIK_url
        ))
len(TIK_data)

2776

In [4]:
# [(region_name, TIK_name, UIK_page_url)]
UIK_links = []
for (region_name, TIK_name, TIK_url) in tqdm_notebook(TIK_data):
    # /html/body/table[2]/tbody/tr[2]/td/a
    TIK_page = requests.get(TIK_url)
    tree = lxml.html.fromstring(TIK_page.text)
    UIKS_page_link = tree.xpath('//table[2]//tr[2]//td//a')
    if len(UIKS_page_link) == 1:
        UIK_page_link = UIKS_page_link[0].get('href')
        UIK_links.append(
            (region_name, TIK_name, UIK_page_link)
        )
len(UIK_links)

2776

In [2]:
# Checkpoint

import pandas as pd

# Save
# df = pd.DataFrame(UIK_links)
# df.to_csv('UIK_links.csv', header=False, index=False)

# Load
df = pd.read_csv('UIK_links.csv')
UIK_links = df.values

In [3]:
# Parse UIK_page
index = 0
bad_UIKs = []
for (region_name, TIK_name, UIK_page_url) in tqdm_notebook(UIK_links):
    try:
        UIK_page =      requests.get(UIK_page_url)
        UIK_page_tree = lxml.html.fromstring(UIK_page.text)
        UIK_names =     UIK_page_tree.xpath('//table[6]//td[2]//tr[1]')
        UIK_table =     UIK_page_tree.xpath('//table[6]//td[2]//tr')

        table = []

        # Get all UIK names
        UIK_names_list = []
        for UIK_name in UIK_names[0].getchildren():
            UIK_names_list.append(UIK_name.text_content())

        n_rows = len(UIK_names_list)
        table += [[region_name] * n_rows]
        table += [[TIK_name]  * n_rows]
        table += [UIK_names_list]

        # Get table content
        counter = 0
        for tr in UIK_table:
            this_row = []
            if tr.get('valign'):
                continue
            for td in tr.getchildren():
                if len(td.getchildren()[0].getchildren()) == 0:
                    this_row += ['empty_row']
                    break
                this_row += [int(td.getchildren()[0].getchildren()[0].text_content())]
            if this_row != ['empty_row']:
                table.append(this_row)
        df = pd.DataFrame(np.array(table).T)
        index += 1
        df.to_csv(f'reports/uik_{index}.csv', header = False, index = False)
    except:
        print(f'Count {len(bad_UIKs) + 1} bad UIKs.')
        bad_UIKs.append(
            (region_name, TIK_name, UIK_page_url)
        )

In [24]:
# Check whether rows are being replaced

from collections import defaultdict

titles_dict = defaultdict(list)

for (region_name, TIK_name, UIK_page_url) in tqdm_notebook(UIK_links):
    UIK_page =      requests.get(UIK_page_url)
    UIK_page_tree = lxml.html.fromstring(UIK_page.text)
    # //table[3]//tr[4]//table[6]//td[1]//table
    UIK_titles =    UIK_page_tree.xpath('//table[3]//tr[4]//table[6]//td[1]//table')
    titles = []
    try:
        for el in UIK_titles[0].getchildren():
            titles.append(el.getchildren()[1].text_content())
    except:
        continue
    for (index, el) in enumerate(titles):
        if index not in titles_dict[el]:
            titles_dict[el].append(index)
titles_dict

defaultdict(list,
            {'Сумма': [0],
             'Число избирателей, включенных в список избирателей ': [1],
             'Число избирательных бюллетеней, полученных участковой избирательной комиссией': [2],
             'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно': [3],
             'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования': [4],
             'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования': [5],
             'Число погашенных избирательных бюллетеней': [6],
             'Число избирательных бюллетеней в переносных ящиках для голосования': [7],
             'Число бюллетеней в стационарных ящиках для голосования': [8],
             'Число недействительных избирательных бюллетеней': [9],
             'Число действительных избирательных бюллетеней': [10],
             'Число утраченных избирательных бюллетеней': [11],
             'Число избирательных б

In [6]:
titles_dict

defaultdict(list,
            {'Сумма': [0],
             'Число избирателей, включенных в список избирателей ': [1],
             'Число избирательных бюллетеней, полученных участковой избирательной комиссией': [2],
             'Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно': [3],
             'Число избирательных бюллетеней, выданных в помещении для голосования в день голосования': [4],
             'Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования': [5],
             'Число погашенных избирательных бюллетеней': [6],
             'Число избирательных бюллетеней в переносных ящиках для голосования': [7],
             'Число бюллетеней в стационарных ящиках для голосования': [8],
             'Число недействительных избирательных бюллетеней': [9],
             'Число действительных избирательных бюллетеней': [10],
             'Число утраченных избирательных бюллетеней': [11],
             'Число избирательных б

In [25]:
(region_name, TIK_name, UIK_page_url) = UIK_links[0]
UIK_page =      requests.get(UIK_page_url)
UIK_page_tree = lxml.html.fromstring(UIK_page.text)
# //table[3]//tr[4]//table[6]//td[1]//table
UIK_titles =    UIK_page_tree.xpath('//table[3]//tr[4]//table[6]//td[1]//table')
titles = []
for el in UIK_titles[0].getchildren():
    titles.append(el.getchildren()[1].text_content())

del titles[13]
del titles[0]
table_titles = ['Регион', 'ТИК', 'УИК'] + titles

In [27]:
from os import listdir

with open('UIK_table.csv', mode='w+', encoding='utf8') as f:
#     with open('reports/header.csv', mode='r', encoding='utf8') as header:
#         line = header.readlines()[0]
#         f.write(line)
    for file in tqdm_notebook(listdir('reports/')):
        if file == 'header.csv':
            continue
        with open('reports/' + file, mode='r', encoding='utf8') as report:
            for line in report:
                f.write(line)

In [54]:
df = pd.read_csv('UIK_table.csv')
df

,Регион,ТИК,УИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Республика Адыгея (Адыгея),Адыгейская,УИК №8000,5297,11000,694,4557,46,5703,740,...,0,0,46,159,80,4779,30,12,5,11
1,Республика Адыгея (Адыгея),Адыгейская,УИК №8001,3287,3600,0,3235,52,313,52,...,0,0,18,32,22,3062,12,4,1,0
2,Республика Адыгея (Адыгея),Адыгейская,УИК №8002,2995,9000,265,2713,17,6005,282,...,0,0,19,122,68,2607,15,6,3,6
3,Республика Адыгея (Адыгея),Адыгейская,УИК №8003,1819,2700,0,1807,12,881,12,...,0,0,12,50,27,1597,10,8,1,3
4,Республика Адыгея (Адыгея),Адыгейская,УИК №8004,3718,8500,0,3712,6,4780,6,...,2,0,25,93,48,3334,25,3,5,8
5,Республика Адыгея (Адыгея),Адыгейская,УИК №8005,2887,8000,187,2691,8,5114,195,...,0,0,13,52,30,2762,12,6,4,1
6,Республика Адыгея (Адыгея),Адыгейская,УИК №8006,2041,2700,201,1746,94,659,295,...,0,0,13,21,20,1937,15,5,2,2
7,Республика Адыгея (Адыгея),Адыгейская,УИК №8007,1172,1800,747,411,14,628,761,...,0,0,5,130,68,890,25,8,1,2
8,Республика Адыгея (Адыгея),Адыгейская,УИК №8008,2700,2700,1365,1175,160,0,1525,...,0,0,8,144,180,2220,37,11,6,6
9,Республика Адыгея (Адыгея),Адыгейская,УИК №8009,203,270,15,188,0,67,15,...,0,0,4,26,7,143,14,0,2,5


In [67]:
url = TIK_data[0][2]
# /html/body/table[2]/tbody/tr[2]/td/a
TIK_page = requests.get(url)
tree = lxml.html.fromstring(region_page.text)
UIKS_page_link = tree.xpath('//table[2]//tr[2]//td//a')
UIKS_page_link[0].get('href'), url

('http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=true&root=1000086&tvd=100100084849217&vrn=100100084849062&prver=0&pronetvd=null&region=99&sub_region=99&type=227&vibid=100100084849217',
 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100084849067&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000364371&type=227')